## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [36]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [37]:
# View the column names in the data
df.columns




Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [38]:
# Use the describe function to gather some basic statistics
df.describe()



,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [39]:
# Use this space to do any additional research
# and familiarize yourself with the data.





In [40]:
# What three item categories had the most entries?
df.value_counts("category").head(3)



category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [41]:
# For the category with the most entries,
# which subcategory had the most entries?
df.value_counts("subcategory").head(1)


subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [42]:
# Which five clients had the most entries in the data?
df.value_counts("client_id").head()


client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [43]:
# Store the client ids of those top 5 clients in a list.
client_id = df["client_id"].value_counts().head().index.tolist()
client_id


[33615, 66037, 46820, 38378, 24741]

In [44]:
# How many total units (the qty column) did the
# client with the most entries order order?
total_units = df[df["client_id"] == 33615]
total_qty = total_units["qty"].sum()
total_qty 



64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [45]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df["line_subtotal"] = df.apply(lambda row: row["unit_price"] * row["qty"], axis = 1)
df[["unit_price", "qty",  "line_subtotal"]].head(2)


,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95


In [46]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def total_weight(row):
    return row["unit_weight"] * row["qty"]
df["total_weight"] = df.apply(total_weight, axis = 1)
def shipping_price(row):
    if row ["total_weight"] > 50: 
        return row["total_weight"] * 7
    elif row["unit_weight"] <= 50:
        return row ["total_weight"] * 10

df["shipping_price"] = df.apply(shipping_price, axis = 1)
df[["unit_weight" , "unit_price" , "qty" , "line_subtotal" , "total_weight" , "shipping_price"]].head()

,unit_weight,unit_price,qty,line_subtotal,total_weight,shipping_price
0,7.50,1096.80,105,115164.00,787.50,5512.50
1,1.49,24.95,21,523.95,31.29,312.90
2,1.68,13.52,39,527.28,65.52,458.64
3,1.23,36.42,29,1056.18,35.67,356.70
4,46.43,195.10,20,3902.00,928.60,6500.20


In [47]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
def line_price(row):
    tax = (row ["line_subtotal"] + row ["shipping_price"]) * .0925
    return tax + row ["line_subtotal"] + row ["shipping_price"]
df["line_price"] = df.apply(line_price, axis =1) 
df["line_price"] = df["line_price"].apply(lambda x : round(x,2))
df[["line_subtotal" , "shipping_price" , "line_price"]].head(3)




,line_subtotal,shipping_price,line_price
0,115164.00,5512.50,131839.08
1,523.95,312.90,914.26
2,527.28,458.64,1077.12


In [48]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
def linecost(row): 
    return row["qty"] * row["unit_cost"] + row["shipping_price"]
df["line_cost"] = df.apply(linecost , axis = 1)
df.head(3)



,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18


In [49]:
# Create a column for the profit of
# each line using line cost and line price
def profitcost(row):
    return row["line_price"] - row["line_cost"]
df["line_profit"] = df.apply(profitcost , axis = 1)
df.head(3)

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.94


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [50]:
# Check your work using the totals above
mostone = df[df["order_id"] == 2742071]
priceone = mostone["line_price"].sum()
priceone = format(priceone, ".2f")

mosttwo = df[df["order_id"] == 2173913]
pricetwo = mosttwo["line_price"].sum()
pricetwo = format(pricetwo, ".2f")

mostthree = df[df["order_id"] == 6128929]
pricethree = mostthree["line_price"].sum()
pricethree = format(pricethree, ".2f")

print(f"Order 2742071 total: ${priceone} \nOrder 2173913 total: ${pricetwo} \nOrder 6128929 total: ${pricethree}")

Order 2742071 total: $152811.89 
Order 2173913 total: $162388.71 
Order 6128929 total: $923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [51]:
# How much did each of the top 5 clients by quantity spend? 
# Check your work from Part 1 for client ids.
topgive = df.groupby("client_id")["line_price"].sum()
totalprice = topgive[client_id]
totalprice




client_id
33615     8377308.53
66037    10259514.80
46820     9743794.33
38378    12906550.87
24741    82268892.04
Name: line_price, dtype: float64

In [52]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.
summary = df[df["client_id"].isin(client_id)]
summary_df = summary.groupby("client_id").agg({"qty":"sum", "shipping_price":"sum", "line_price":"sum", "line_profit":"sum"}).reset_index(drop=False)
summary_df = summary_df.sort_values(by=["line_profit"], ascending=False)
summary_df

,client_id,qty,shipping_price,line_price,line_profit
0,24741,239862,5126448.37,82268892.04,36579992.33
2,38378,73667,3429455.40,12906550.87,3271829.89
4,66037,43018,1395151.85,10259514.80,3255031.82
3,46820,75768,1601448.84,9743794.33,2736602.69
1,33615,64313,1828984.89,8377308.53,2201994.62


In [53]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.
def millions(value):
    return value/1000000
summary_df["line_profit"] = summary_df["line_profit"].apply(millions)
summary_df["shipping_price"] = summary_df["shipping_price"].apply(millions)
summary_df["line_price"] = summary_df["line_price"].apply(millions)
summary_df
def millionformat(value):
    return "${:.2f}M".format(value)
summary_df["line_profit"] = summary_df["line_profit"].apply(millionformat)
summary_df["shipping_price"] = summary_df["shipping_price"].apply(millionformat)
summary_df["line_price"] = summary_df["line_price"].apply(millionformat)
summary_df = summary_df.rename(columns={
    "client_id":"Client ID",
    "qty":"Units",
    "shipping_price":"Shipping Price",
    "line_price":"Total Revenue",
    "line_profit":"Total Profit"})
summary_df


,Client ID,Units,Shipping Price,Total Revenue,Total Profit
0,24741,239862,$5.13M,$82.27M,$36.58M
2,38378,73667,$3.43M,$12.91M,$3.27M
4,66037,43018,$1.40M,$10.26M,$3.26M
3,46820,75768,$1.60M,$9.74M,$2.74M
1,33615,64313,$1.83M,$8.38M,$2.20M


In [54]:
# Sort the updated data by "Total Profit" form highest to lowest
summary_df = summary_df.sort_values(by=["line_profit"], ascending=False)
summary_df 

KeyError: 'line_profit'

In [55]:
#Challenge Summary- 
#We have organized and transformed the data to show that 66037 did indeed spend more than 33615.
#We used the list from part one to determine and confirm that the calculations match the receipts.
#Used new columns and confirmed values to find information related to top 4 client qty and spend.  
#Checked work for errors and functionality.